<a href="https://colab.research.google.com/github/dolmani38/Summary2/blob/main/%ED%95%9C%EA%B8%80%EB%AC%B8%EC%9E%A5%EC%88%98%EC%A7%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 한글 뉴스 크롤러

In [1]:
if True:
    from google.colab import drive
    drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import sys
import requests
import pandas as pd
import datetime
import requests
import re
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook
from multiprocessing import Pool

client_id = "waXxTjNdGjzj8xtT3nbF" #1.에서 취득한 아이디 넣기
client_secret = "H6mejnkuYx"  #1. 에서 취득한 키 넣기


# 간단한 전처리
def clean_text(txt):
    txt = txt.replace('\n',' ')
    txt = txt.replace('\r',' ')    
    txt = txt.replace('=','')
    txt = txt.replace('\"','')   
    txt = txt.replace('\'','')
    txt = txt.replace(',','')
    txt = txt.replace('..','')
    txt = txt.replace('...','')
    txt = txt.replace('”','')
    txt = txt.replace('.','. ')
    txt = txt.replace('  ',' ')
    txt = txt.replace('  ',' ')    
    txt = txt.replace('  ',' ')   
    txt = txt.replace('  ',' ')           
    txt = txt.replace('  ',' ')
    txt = txt.replace('  ',' ')    
    txt = txt.replace('  ',' ')   
    txt = txt.replace('  ',' ')             
    return txt.strip()

def clean_html(x):
    x = re.sub("\&\w*\;","",x)
    x = re.sub("<.*?>","",x)
    return x

def get_news_urls(search_word = '코로나',max_display = 100):
    encode_type = 'json' #출력 방식 json 또는 xml
    #max_display = 100 #출력 뉴스 수
    sort = 'date' #결과값의 정렬기준 시간순 date, 관련도 순 sim
    start = 1 # 출력 위치

    url = f"https://openapi.naver.com/v1/search/news.{encode_type}?query={search_word}&display={str(int(max_display))}&start={str(int(start))}&sort={sort}"

    #print(url)
    #헤더에 아이디와 키 정보 넣기
    headers = {'X-Naver-Client-Id' : client_id,
            'X-Naver-Client-Secret':client_secret
            }

    df = None
    #HTTP요청 보내기
    r = requests.get(url, headers=headers)
    if r.status_code == 200:
        #요청 결과 보기 200 이면 정상적으로 요청 완료
        df = pd.DataFrame(r.json()['items'])
        df['title'] = df['title'].apply(lambda x: clean_html(x))
        df['description'] = df['description'].apply(lambda x: clean_html(x))
    return df

def get_news_title_contents(url):
    headers = {'User-Agent':'Chrome/66.0.3359.181'}
    main_news = []
    result = requests.get(url,headers=headers)
    if result.status_code == 200:
        bs_obj = BeautifulSoup(result.content, "html.parser")
        #print(bs_obj)
        try:
            title = bs_obj.find("h3",{"id":"articleTitle"}).text
            body = bs_obj.find("div",{"class":"_article_body_contents"}).get_text("\n").replace("// flash 오류를 우회하기 위한 함수 추가","").replace("function _flash_removeCallback() {}","")
            #aid = bs_obj.find("div",{"class":"article_header"}).find("img")["title"]
            main_news = [clean_text(title),clean_text(body)]
        except:
            pass
            #print("Unexpected error:", sys.exc_info()[0])
    return main_news    

In [3]:
def make_corpus(search_word = '코로나'):
    f = open("/content/drive/MyDrive/Colab Notebooks/summary/korean_news_corpus.csv", "a")
    df = get_news_urls(search_word = search_word,max_display = 100)
    if df is not None:
        for url in df['link']:
            if url.startswith('https://news.naver.com/'):
                #print(url)
                news = get_news_title_contents(url)
                if (len(news) == 2):
                    print('\rtitle:',news[0],end="", flush=True)
                    f.write(news[0] + "," + news[1] + "\r\n")
    f.close() 

In [ ]:
import random
source = """
영국 과학자들이 모든 신종 코로나바이러스 감염증(COVID-10·코로나19)을 유발하는 사스코로나바이러스-2(SARS-CoV-2)의 변이바이러스에 효과가 있는 항바이러스 치료제 후보물질을 개발했다영국 브리스톨대 과학자들이 설립한 바이오기업 헤일로쎄라퓨틱스는 모든 코로나19 변이바이러스에 효과가 있는 치료제 후보물질을 개발하고 조만간 임상에 나선다고 13일(현지시간) 밝혔다코로나바이러스가 인체에 침입할 때 인간 세포 수용체와 결합하는 스파이크 단백질의 모양을 변화시켜 바이러스의 감염 능력을 억제하는 분자를 활용한 항바이러스 치료 후보물질이다브리스톨대 연구진은 앞서 국제학술지 ‘사이언스’에 새로운 코로나19 치료전략을 제시한 논문을 발표했다. 불포화지방산의 일종인 ‘리놀레산’에 코로나바이러스를 노출시키면 바이러스 표면에 돌기 모양으로 생긴 스파이크 단백질이 닫히면서 감염되기 어려운 형태로 바뀐다는 사실을 발견한 연구논문이다임상을 진행하기 전 실험실에서 진행한 연구에서 연구진이 개발한 치료후보물질은 코와 인후, 폐의 세포에 코로나바이러스가 침투하는 것을 막는 것으로 나타났다. 특히 영국과 남아프리카공화국, 브라질 변이바이러스에서도 동일한 효과를 보였다. 헤일로쎄라퓨틱스는 비강 스프레이와 천식 환자들이 사용하는 흡입기 형태로도 치료제 임상을 진행할 계획이다새로운 치료전략을 토대로 한 코로나19 항바이러스 치료제 개발을 위해 브리스톨대 연구진은 생명공학기업 헤일로쎄라퓨틱스를 설립했다. 현재 헤일로쎄라퓨틱스는 치료후보물질의 임상시험을 준비중이다. 임상 승인에 이어 임상에 성공하게 되면 전세계 코로나19 환자가 사용할 수 있는 항바이러스 치료제가 나올 것으로 기대된다헤일로쎄라퓨틱스는 “임상에 성공할 경우 코로나19 환자의 바이러스 증식을 억제하는 것은 물론 감염자와 밀접 접촉한 사람들에 대한 바이러스 추가 전파를 막을 수 있는 항바이러스 치료제로 활용될 수 있을 것으로 예상된다”고 밝혔다치료제 개발 연구를 이끌고 있는 임레 버거 막스플랑크-브리스톨 소형생물학센터 교수는 “우리가 개발한 치료제의 목적은 체내로 유입되는 바이러스의 양을 크게 줄이고 억제하는 것”이라며 “바이러스에 노출되거나 감염돼도 바이러스가 폐나 다른 인체 장기로 증식하는 것을 막을 수 있다”고 말했다애덤 핀 브로스톨의과대학 교수는 “변이바이러스의 위협으로 현재 사용 가능한 백신의 예방 효과가 떨어지고 변이바이러스 재감염 사례가 늘어나는 실질적인 위험이 생길 수 있다”며 “모든 변이바이러스 균주에 걸쳐 효과가 있는 항바이러스 치료제로 백신을 보완할 수 있을 것”이라고 기대했다.
[헤럴드경제=박세환 기자] 미국내 신종 코로나바이러스 감염증(코로나19)의 확산세가 수그러들면서 식당·상점이 문을 여는 등 일상으로의 복귀에 시동을 걸자 총기 난사도 함께 증가하고 있다는 진단이 나와 관심을 모으고 있다.
16일(현지시간) CNN 방송에 따르면 비영리 연구단체 ‘총기폭력저장소(GVA)’의 데이터, 경찰 보고 등을 분석한 결과, 최근 한 달 새 미 전역에서 최소 45건의 총기 난사 사건이 발생한 것으로 집계됐다.
한인 희생자가 4명이나 나온 조지아주(州) 애틀랜타 일원 마사지숍 총격 사건(3월 16일)을 시작으로 이후 44건의 무차별 총격 사건이 꼬리에 꼬리를 물고 벌어진 것이다.
또 GVA에 따르면 올해 들어 미국에서는 147건의 총기 난사 사건이 벌어졌다.
CNN은 총기 난사 사건을 총격범을 제외하고 4명 이상이 총에 맞아 사상한 사건으로 규정했다.
미국에서는 전날인 15일 밤에도 인디애나주(州) 인디애나폴리스의 물류업체 페덱스 창고에서 총격 사건이 벌어져 최소 8명이 숨졌다.
코로나19 사태로 자택 대피령이 내려지고 기업체·식당·가게가 문을 닫은 지난 1년간 비교적 잠잠했던 총기 난사 사건이 경제 정상화와 함께 다시 고개를 드는 것이다.
CNN은 지난달 “미국인들은 1년간 ‘정상으로 돌아가고 싶다’고 말해왔다. 비극적이게도, 그들의 소원이 이뤄지고 있다”고 자조적으로 진단했다.
그러면서 “공공장소로의 점진적 복귀와 함께 이 나라의 모두가 너무도 열렬히 옆으로 치워두고 싶었던 유령이 돌아오고 있다”며 “총기 난사가 신문의 머리기사를 다시 장식하고 있다”고 전했다.
이 매체는 지난 12개월간 식료품점에 가서 감염을 피하기 위해 바짝 경계해야 했던 미국인들에게 총격 사건이 이제는 다른 이유로 여전히 경계해야 한다는 사실을 일깨워주고 있다고 지적했다.
조 바이든 미국 대통령은 16일 성명을 내고 “너무 많은 미국인이 매일 총기 폭력으로 숨진다”며 “그것은 우리의 인격을 더럽히고, 우리나라의 영혼을 찌른다”고 말했다.
그러면서 “우리는 생명을 구하기 위해 더 많은 일을 해야 하고 할 수 있다”며 “우리는 행동해야 한다”고 강조했다.
CNN은 총기 폭력과 관련해 조치가 필요하다는 여론이 높은데도 의회가 총기 규제 강화 법안을 통과하는 데 미온적이라고 비판했다.
퀴니피액대학이 벌인 최근 여론조사에 따르면 미국인의 89%가 모든 총기 구매자들을 상대로 신원 조회를 의무화하는 방안을 지지하는 것으로 나타났다.
또 정신질환자 등 타인을 해코지하거나 자해할 가능성이 있어 위협으로 판단되는 개인으로부터 총기류를 일정 기간 법원이 압류하는 ‘적기법(red flag law)’에 찬성한다는 사람도 75%나 됐고, 과반인 51%는 총기 난사에 자주 쓰이는 고화력 무기인 돌격소총 금지를 지지하는 것으로 나왔다.
크리스 머피(민주·코네티컷) 상원의원은 인디애나폴리스 총격이 벌어진 뒤 이날 트위터에 글을 올려 의회가 행동에 나설 것을 촉구했다.
그는 “총격에 총격이 이어진 뒤에도 의회가 전적으로 아무것도 안 할 때 우리는 이 대량학살에 공모자가 된다”며 “우리의 침묵은 지지로 해석돼왔다”고 썼다.
"""
source = clean_text(source)
source = source.replace(" ","")
source = source.replace(".","")
while(True):
    keyword = random.choice(source)+random.choice(source)
    try:
        make_corpus(search_word = keyword)
    except KeyboardInterrupt as ki:
        raise ki        
    except:
        print("Unexpected error:", sys.exc_info()[0])        

title: 이미선 임명 반대 47. 4%…후보 부적격 47. 1% -리얼미터

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/summary/korean_news_corpus.csv')
df

,title,contents
0,문 대통령 변창흠 국토장관 사의표명 사실상 수용,정만호 국민소통수석이 12일 오후 청와대 춘추관 대브리핑룸에서 변창흠 국토부 장관 ...
1,계급장 수여하는 문 대통령,(아산뉴스1) 이광호 기자 문재인 대통령이 12일 오후 충남 아산시 경찰대학에서 열...
2,계급장 수여하는 문 대통령,(아산뉴스1) 이광호 기자 문재인 대통령이 12일 오후 충남 아산시 경찰대학에서 열...
3,수상자 메달 걸어주는 문 대통령,(아산뉴스1) 이광호 기자 문재인 대통령이 12일 오후 충남 아산시 경찰대학에서 열...
4,정몽구 서울아산병원에 50억 쾌척,인재 양성·소외 계층 지원 등 계획 “부친 질병·가난 악순환 끊기 원해 국내 최고 ...
...,...,...
37227,이런 광경 얼마 만인지…쏟아진 中 관광객에 웃은 면세점,한한령 이후 최대 규모 5000명 인센티브 관광 중국인 관광객 증가 터닝포인트 되길...
37228,文 투기와의 전쟁에서 지지 않을 것… 고강도 규제 예고편?,부동산 시장 안정 실수요자 보호 투기 억제에 대한 정부의 의지는 확고 풍선효과 계속...
37229,그룹 신화 멤버 김동완씨 덕분에 만나게 된 외국인 독자,[내가 만난 독자] 소년의 레시피를 읽고 쓰고 일본어로 옮기는 기쿠치 미유키씨 [오...
37230,[여의도 사사건건] 내일부터 21대 총선 예비후보 등록…선거법 어디로?,- 낙후된 지역 SOC 신규 사업 예산 확보는 잘한 일…그래야 균형 발전 - 4+1...


In [ ]:
df['contents'][100]

'전북서 본부장으로 일할 때 바람직하지 않은 일 했다 3월3일 오후 경남 진주시 충무공동 한국토지주택공사(LH) 본사 앞에 빨간 신호등이 켜 있다. 문재인 대통령은 이날 LH 직원들의 100억원대 광명·시흥 신도시 투기 의혹에 대해 철저한 조사를 지시했다. ⓒ 연합뉴스 직원들의 광명·시흥 3기 신도시 지역 투기 의혹에 휩싸인 한국토지주택공사(LH)의\xa0고위 간부가 12일 스스로 목숨을 끊었다. 경기남부경찰청 등에 따르면 이날 오전 9시40분께 경기도 성남시 분당의 한 아파트 화단에서 LH 현직 고위 간부 A씨가 쓰러진 채로 발견됐다. A씨는 병원으로 이송됐지만 사망했다. \xa0 A씨는 전북에서 본부장으로 근무할 때 바람직하지 않은 일을 했다. 괴롭다. 국민에게 죄송하다는 내용의 유서를 남긴 것으로 전해졌다. 경찰은\xa0아파트 주변 폐쇄(CCTV)회로 등을 조사한 결과 현재까지 범죄 혐의점은 발견되지 않았으며 사망 원인 등에 대한 추가 조사를 벌이고 있다고 밝혔다. \xa0 ※ 우울감 등 말하기 어려운 고민이 있거나 주변에 이런 어려움을 겪는 가족·지인이 있으면 자살 예방 핫라인 ☎1577-0199 희망의 전화 ☎129 생명의 전화 ☎1588-9191 청소년 전화 ☎1388 등에서 24시간 전문가의 상담을 받을 수 있습니다. 이혜영 기자 zero@sisajournal. com <저작권자 ⓒ 시사저널 무단 전재 및 재배포 금지> ☞ 네이버에서 시사저널 뉴스를 받아 보세요 ▶ 시사저널 최신호 보기'

In [ ]:
# 검사...
pattens = ["[34569][0-9]{3}[\;.\;-\; ][0-9]{4}[\;.\;-\; ][0-9]{4}[\;.\;-\; ][0-9]{4}",
           "[0-9]{2,3}[\:\s\;.\;,\;-;)][0-9]{3,4}[\:\s\;.\;,\;-][0-9]{4}",
           "[0-9]{1}[0-9]{1}[\W]?[0-1]{1}[0-9]{1}[\W]?[0-3]{1}[\W]?[0-9]{1}[\W]?[1-4]{1}[\W]?[0-9]{1}[\W]?[0-9]{1}[\W]?[0-9]{1}[\W]?[0-9]{1}[\W]?[0-9]{1}[\W]?[0-9]{1}",
           "[0-9]{2}[\:\s\;.\;,\;-][0-9]{2}[\:\s\;.\;,\;-][0-9]{6}|[0-9]{3}[\:\s\;.\;,\;-]([0-9]{5,6}[\:\s\;.\;,\;-][0-9]{3}|[0-9]{6}[\:\s\;.\;,\;-][0-9]{5}|[0-9]{2,3}[\:\s\;.\;,\;-][0-9]{6}|[0-9]{2}[\:\s\;.\;,\;-][0-9]{7}|[0-9]{2}[\:\s\;.\;,\;-][0-9]{4,6}[\:\s\;.\;,\;-][0-9]|[0-9]{5}[\:\s\;.\;,\;-][0-9]{3}[\:\s\;.\;,\;-][0-9]{2}|[0-9]{2}[\:\s\;.\;,\;-][0-9]{5}[\:\s\;.\;,\;-][0-9]{3}|[0-9]{4}[\:\s\;.\;,\;-][0-9]{4}[\:\s\;.\;,\;-][0-9]{3}|[0-9]{6}[\:\s\;.\;,\;-][0-9]{2}[\:\s\;.\;,\;-][0-9]{3}|[0-9]{2}[\:\s\;.\;,\;-][0-9]{2}[\:\s\;.\;,\;-][0-9]{7})|[0-9]{4}[\:\s\;.\;,\;-]([0-9]{3}[\:\s\;.\;,\;-][0-9]{6}|[0-9]{2}[\:\s\;.\;,\;-][0-9]{6}[\:\s\;.\;,\;-][0-9])|[0-9]{5}[\:\s\;.\;,\;-][0-9]{2}[\:\s\;.\;,\;-][0-9]{6}|[0-9]{6}[\:\s\;.\;,\;-][0-9]{2}[\:\s\;.\;,\;-][0-9]{5,6}"
           ]

filters = []
for p in pattens:
    filters.append(re.compile(p))


In [ ]:
sentences = []

for raw_text in df['contents']:
    try:
        docs = nltk.sent_tokenize(clean_text(raw_text))
        for txt in docs:
            if txt.find('▶') > -1 or txt.find('@') > -1 or txt.find('ⓒ') > -1: 
                pass
            else:
                txt = txt.strip()
                if len(txt) > 40:
                    #ok = True
                    #for f in filters:
                    #    if len(f.findall(txt)) > 0:
                    #        ok = False
                    #        print('fitered:',txt)
                    #if ok:
                    if any(chr.isdigit() for chr in txt):
                        pass
                    else:
                        sentences.append(txt)
    except KeyboardInterrupt as ki:
        raise ki        
    except:
        print("Unexpected error:", sys.exc_info()[0])                         

Unexpected error: <class 'AttributeError'>
Unexpected error: <class 'AttributeError'>
Unexpected error: <class 'AttributeError'>
Unexpected error: <class 'AttributeError'>
Unexpected error: <class 'AttributeError'>
Unexpected error: <class 'AttributeError'>
Unexpected error: <class 'AttributeError'>


In [ ]:
len(sentences)

407555

In [ ]:
sentences[254157]

'축산물 공판장에서 무더기로 확진자가 나온 데다 어린이집 병원 목욕탕·사우나 제조업장 등에서도 신규 감염이 잇따르고 있습니다.'

In [ ]:
df1 = pd.DataFrame({'sentence':sentences})

In [ ]:
df1

,sentence
0,연합뉴스 문재인 대통령이 변창흠 국토교통부 장관의 사의 표명을 사실상 수용했다.
1,문 대통령은 변 장관의 사의 표명에 책임지는 모습을 보일 수밖에 없다고 밝혔다.
2,이에 앞서 변창흠 국토교통부 장관은 LH 땅 투기 의혹 사건과 관련한 책임론에 대해...
3,변 장관은 이날 국회 국토교통위원회 전체회의에 참석해 “LH 사태로 국민들이 걱정하...
4,인재 양성·소외 계층 지원 등 계획 “부친 질병·가난 악순환 끊기 원해 국내 최고 ...
...,...
407550,마무리를 해야 하는데 만약에 선거법 올린다고 해도 필리버스터가 있기 때문에 쉽지 않...
407551,의사 진행권은 의장한테 있기 때문에 회기 정하는 것은 필리버스터에 적용 안 된다고 ...
407552,JC 중앙회장도 하고 굉장히 똑똑한 친구인데 아버지 뒤를 이어서 자기가 아버지가 공...
407553,그리고 의정부에서 출마한다는데 의정부 시민들도 문희상 의장의 아들이 출마하는지 안 ...


In [ ]:
df1.to_csv('/content/drive/MyDrive/Colab Notebooks/summary/korean_sentences.csv')